In [1]:
# %run main --data './data/city' --epochs 10
%run main --data './data/city' --emsize 650 --nhid 650 --dropout 0.5 --epochs 40 --tied

-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 31.85s | valid loss  2.44 | valid ppl    11.45
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   2 | time: 39.84s | valid loss  1.34 | valid ppl     3.82
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   3 | time: 43.92s | valid loss  1.21 | valid ppl     3.34
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   4 | time: 45.76s | valid loss  1.11 | valid ppl     3.04
--------------------------------------------------------------------------

In [8]:
import numpy as np

In [9]:
def predict_next_city(previous):
    model = None
    with open('./model.pt', 'rb') as f:
        model = torch.load(f).to(device)
    model.eval()    
    hidden = model.init_hidden(1)
    input = None
    next_city = []

    with torch.no_grad():
        for p in previous[:-1]:
            index = corpus.dictionary.word2idx[p]
            input = torch.randint(index,index+1, (1, 1), dtype=torch.long).to(device)
            _, hidden = model(input, hidden)
            
        index = corpus.dictionary.word2idx[previous[-1]]
        input = torch.randint(index,index+1, (1, 1), dtype=torch.long).to(device)            
        while True:
            output, hidden = model(input, hidden)
            word_weights = output.squeeze().div(1.0).exp().cpu()
            word_idx = torch.multinomial(word_weights, 1)[0]
            input.fill_(word_idx)
            word = corpus.dictionary.idx2word[word_idx]
            if word == '<eos>':
                if len(next_city)==0:
                    continue
                else:
                    break
            next_city.append(word)
                
        return next_city

In [10]:
test_lines = None
train_lines = None

with open('./data/city/test.txt') as f:
    test_lines = f.readlines()
    
with open('./data/city/train.txt') as f:
    train_lines = f.readlines()    

In [11]:
def random_query(lines):
    line = np.random.choice(lines)
    query = line.split('  ')[0].split()
    true = line.split('  ')[-1][:-1]
    pred = ' '.join(predict_next_city(query))
    print('Query:',' '.join(query))
    print('     =',true)
    print('     >',pred)
    print(true in pred)

In [22]:
random_query(train_lines)

Query: San Antonio TX, Austin TX,
     = Houston TX,
     > Chicago IL, Milwaukee WI,
False


In [23]:
total = 0
correct = 0

for line in train_lines:
    query = line.split('  ')[0].split()
    true = line.split('  ')[-1][:-1]
    pred = ' '.join(predict_next_city(query))
    if true in pred:
        correct += 1
    total +=1

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800


In [24]:
correct*100.0 / total

12.920203735144312